In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin

# Set the search query for apple seeds
search_query = "Single MRI image duchenne muscular dystrophy"

# Set the number of images to download
num_images = 100

# Set the directory to save the downloaded images
save_directory = "/content/drive/MyDrive/Neuro/dhdgen"

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Prepare the search URL
search_url = f"https://www.google.com/search?q={search_query}&tbm=isch"

# Send a GET request to the search URL
response = requests.get(search_url)

# Parse the response content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all image elements in the parsed HTML
image_elements = soup.find_all('img')

# Download the images
downloaded_images = 0
for i, img in enumerate(image_elements):
    # Get the image source URL
    img_url = img['src']

    try:
        # Check if the URL is a relative URL
        if img_url.startswith('/'):
            # Construct the absolute URL using the base URL of the search page
            img_url = urljoin(search_url, img_url)

        # Send a GET request to download the image
        img_response = requests.get(img_url, stream=True)
        img_response.raise_for_status()
        content_type = img_response.headers.get('Content-Type')
        if content_type == 'image/jpeg':
            file_extension = 'jpg'
        elif content_type == 'image/png':
            file_extension = 'png'
        else:
            continue
        # Save the image to the specified directory
        with open(os.path.join(save_directory, f"image{i+1}.jpg"), 'wb') as f:
            for chunk in img_response.iter_content(chunk_size=128):
                f.write(chunk)

        downloaded_images += 1

        if downloaded_images >= num_images:
            break

    except requests.exceptions.HTTPError:
        continue

# Print the number of downloaded images
print("Number of downloaded images:", downloaded_images)

Number of downloaded images: 20


In [ ]:
import os
from sklearn.model_selection import train_test_split

# Define the path to your image dataset directory
dataset_dir = '/content/drive/MyDrive/Neuro/dhdgen'

# List all image file names in the dataset directory
image_files = [file for file in os.listdir(dataset_dir) if file.endswith('.jpg')]  # Adjust the extension if needed

# Split the dataset into training and testing sets
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Define directories for train and test sets
train_dir = '/content/drive/MyDrive/Neuro/dhdgen/test'
test_dir = '/content/drive/MyDrive/Neuro/dhdgen/train'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Move files to train and test directories
for file in train_files:
    src_path = os.path.join(dataset_dir, file)
    dst_path = os.path.join(train_dir, file)
    os.rename(src_path, dst_path)

for file in test_files:
    src_path = os.path.join(dataset_dir, file)
    dst_path = os.path.join(test_dir, file)
    os.rename(src_path, dst_path)

print(f"Number of images in train set: {len(train_files)}")
print(f"Number of images in test set: {len(test_files)}")

Number of images in train set: 16
Number of images in test set: 4


In [ ]:
# Multi-Layer Perceptron (MLP)

from google.colab import drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import os
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define main data directory
neuro_dir = '/content/drive/MyDrive/Neuro'

# Create an MLP model
model = Sequential([
    Flatten(input_shape=(224, 224, 3)),
    Dense(128, activation='relu'),
    Dropout(0.5),  # This line had an indentation issue in your previous message
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# List all main subfolders
main_subfolders = ['dhdleg', 'dhdhead', 'dhdgen', 'dhdpelvic', 'llg']

# Initialize counters for high severe and less severe images
high_severe_count = 0
less_severe_count = 0

# Lists to store true labels and predicted probabilities for evaluation
true_labels = []
predicted_probs = []

# Loop through main subfolders
for main_subfolder in main_subfolders:
    main_data_dir = os.path.join(neuro_dir, main_subfolder)

    subsubfolders = ['train', 'test']

    for subsubfolder in subsubfolders:
        data_dir = os.path.join(main_data_dir, subsubfolder)

        print(f"Processing subsubfolder: {main_subfolder}/{subsubfolder}")

        # List all image filenames in the subfolder
        image_filenames = [filename for filename in os.listdir(data_dir) if filename.lower().endswith(('.jpg', '.jpeg'))]

        if not image_filenames:
            print("No images found in the directory.")
            continue

        for image_filename in image_filenames:
            # Load the image, preprocess it, and make a prediction
            image_path = os.path.join(data_dir, image_filename)
            img = load_img(image_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values

            true_label = 1  # Assuming all are high severe
            true_labels.append(true_label)

            prediction = model.predict(np.expand_dims(img_array, axis=0))
            predicted_prob = prediction[0][0]
            predicted_probs.append(predicted_prob)

            if predicted_prob <= 0.40:
                less_severe_count += 1
            else:
                high_severe_count += 1

            severity = "less severe" if predicted_prob <= 0.70 else "high severe"

            print(f"Image: {image_filename} - Predicted Accuracy: {predicted_prob:.4f} - Severity: {severity}")

# Calculate overall accuracies
total_images = high_severe_count + less_severe_count
high_severe_accuracy = (high_severe_count / total_images) * 100 if total_images > 0 else 0
less_severe_accuracy = (less_severe_count / total_images) * 100 if total_images > 0 else 0

print(f"\nOverall High Severe Accuracy: {high_severe_accuracy:.2f}%")
print(f"Overall Less Severe Accuracy: {less_severe_accuracy:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing subsubfolder: dhdleg/train
1/1 [==============================] - 0s 92ms/step
Image: image5.jpg - Predicted Accuracy: 0.5048 - Severity: less severe
1/1 [==============================] - 0s 40ms/step
Image: image21.jpg - Predicted Accuracy: 0.5271 - Severity: less severe
1/1 [==============================] - 0s 36ms/step
Image: image16.jpg - Predicted Accuracy: 0.3855 - Severity: less severe
1/1 [==============================] - 0s 34ms/step
Image: image8.jpg - Predicted Accuracy: 0.3617 - Severity: less severe
1/1 [==============================] - 0s 40ms/step
Image: image13.jpg - Predicted Accuracy: 0.4662 - Severity: less severe
1/1 [==============================] - 0s 33ms/step
Image: image10.jpg - Predicted Accuracy: 0.4506 - Severity: less severe
1/1 [==============================] - 0s 34ms/step
Image: image7.jpg - Predicted Accuracy:

In [ ]:
# rnn

from google.colab import drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, SimpleRNN, Dropout, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import os
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define main data directory
neuro_dir = '/content/drive/MyDrive/Neuro'

# Load pre-trained InceptionV3 model without top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Reshape for RNN
rnn_input = Reshape((1, 128))(x)  # Reshape to (timesteps, features)

# Add RNN layer
rnn_layer = SimpleRNN(64)(rnn_input)

predictions_layer = Dense(1, activation='sigmoid')(rnn_layer)
model = Model(inputs=base_model.input, outputs=predictions_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# List all main subfolders
main_subfolders = ['dhdleg', 'dhdhead', 'dhdgen', 'dhdpelvic','llg']

# Initialize counters for high severe and less severe images
high_severe_count = 0
less_severe_count = 0

# Lists to store true labels and predicted probabilities for evaluation
true_labels = []
predicted_probs = []

# Loop through main subfolders
for main_subfolder in main_subfolders:
    main_data_dir = os.path.join(neuro_dir, main_subfolder)

    subsubfolders = ['train', 'test']

    for subsubfolder in subsubfolders:
        data_dir = os.path.join(main_data_dir, subsubfolder)

        print(f"Processing subsubfolder: {main_subfolder}/{subsubfolder}")

        # List all image filenames in the subfolder
        image_filenames = [filename for filename in os.listdir(data_dir) if filename.lower().endswith(('.jpg', '.jpeg'))]

        if not image_filenames:
            print("No images found in the directory.")
            continue

        for image_filename in image_filenames:
            # Load the image, preprocess it, and make a prediction
            image_path = os.path.join(data_dir, image_filename)
            img = load_img(image_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values
            img_array = img_array.reshape((1,) + img_array.shape)

            true_label = 1  # Assuming all are high severe
            true_labels.append(true_label)

            prediction = model.predict(img_array)
            predicted_prob = prediction[0][0]
            predicted_probs.append(predicted_prob)

            if predicted_prob <= 0.40:
                less_severe_count += 1
            else:
                high_severe_count += 1

            severity = "less severe" if predicted_prob <= 0.40 else "high severe"

            print(f"Image: {image_filename} - Predicted Accuracy: {predicted_prob:.4f} - Severity: {severity}")

# Calculate overall accuracies
total_images = high_severe_count + less_severe_count
high_severe_accuracy = (high_severe_count / total_images) * 100 if total_images > 0 else 0
less_severe_accuracy = (less_severe_count / total_images) * 100 if total_images > 0 else 0

print(f"\nOverall High Severe Accuracy: {high_severe_accuracy:.2f}%")
print(f"Overall Less Severe Accuracy: {less_severe_accuracy:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing subsubfolder: dhdleg/train
1/1 [==============================] - 3s 3s/step
Image: image5.jpg - Predicted Accuracy: 0.5851 - Severity: high severe
1/1 [==============================] - 0s 259ms/step
Image: image21.jpg - Predicted Accuracy: 0.4938 - Severity: high severe
1/1 [==============================] - 0s 185ms/step
Image: image16.jpg - Predicted Accuracy: 0.4414 - Severity: high severe
1/1 [==============================] - 0s 153ms/step
Image: image8.jpg - Predicted Accuracy: 0.3792 - Severity: less severe
1/1 [==============================] - 0s 162ms/step
Image: image13.jpg - Predicted Accuracy: 0.5560 - Severity: high severe
1/1 [==============================] - 0s 147ms/step
Image: image10.jpg - Predicted Accuracy: 0.4847 - Severity: high severe
1/1 [==============================] - 0s 153ms/step
Image: image7.jpg - Predicted Accur

In [11]:
#cnn

from google.colab import drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, SimpleRNN, Dropout, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import os
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define main data directory
neuro_dir = '/content/drive/MyDrive/Neuro'

# Load pre-trained InceptionV3 model without top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Reshape for RNN
rnn_input = Reshape((1, 128))(x)  # Reshape to (timesteps, features)

# Add RNN layer
rnn_layer = SimpleRNN(64)(rnn_input)

predictions_layer = Dense(1, activation='sigmoid')(rnn_layer)
model = Model(inputs=base_model.input, outputs=predictions_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# List all main subfolders
main_subfolders = ['dhdleg', 'dhdhead', 'dhdgen', 'dhdpelvic','llg']

# Initialize counters for high severe and less severe images
high_severe_count = 0
less_severe_count = 0

# Lists to store true labels and predicted probabilities for evaluation
true_labels = []
predicted_probs = []

# Loop through main subfolders
for main_subfolder in main_subfolders:
    main_data_dir = os.path.join(neuro_dir, main_subfolder)

    subsubfolders = ['train', 'test']

    for subsubfolder in subsubfolders:
        data_dir = os.path.join(main_data_dir, subsubfolder)

        print(f"Processing subsubfolder: {main_subfolder}/{subsubfolder}")

        # List all image filenames in the subfolder
        image_filenames = [filename for filename in os.listdir(data_dir) if filename.lower().endswith(('.jpg', '.jpeg'))]

        if not image_filenames:
            print("No images found in the directory.")
            continue

        for image_filename in image_filenames:
            # Load the image, preprocess it, and make a prediction
            image_path = os.path.join(data_dir, image_filename)
            img = load_img(image_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values
            img_array = img_array.reshape((1,) + img_array.shape)

            true_label = 1  # Assuming all are high severe
            true_labels.append(true_label)

            prediction = model.predict(img_array)
            predicted_prob = prediction[0][0]
            predicted_probs.append(predicted_prob)

            if predicted_prob <= 0.60:
                less_severe_count += 1
            else:
                high_severe_count += 1

            severity = "less severe" if predicted_prob <= 0.60 else "high severe"

            print(f"Image: {image_filename} - Predicted Accuracy: {predicted_prob:.4f} - Severity: {severity}")

# Calculate overall accuracies
total_images = high_severe_count + less_severe_count
high_severe_accuracy = (high_severe_count / total_images) * 100 if total_images > 0 else 0
less_severe_accuracy = (less_severe_count / total_images) * 100 if total_images > 0 else 0

print(f"\nOverall High Severe Accuracy: {high_severe_accuracy:.2f}%")
print(f"Overall Less Severe Accuracy: {less_severe_accuracy:.2f}%")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing subsubfolder: dhdleg/train
1/1 [==============================] - 2s 2s/step
Image: image5.jpg - Predicted Accuracy: 0.4466 - Severity: less severe
1/1 [==============================] - 0s 163ms/step
Image: image21.jpg - Predicted Accuracy: 0.3682 - Severity: less severe
1/1 [==============================] - 0s 179ms/step
Image: image16.jpg - Predicted Accuracy: 0.6172 - Severity: high severe
1/1 [==============================] - 0s 157ms/step
Image: image8.jpg - Predicted Accuracy: 0.5338 - Severity: less severe
1/1 [==============================] - 0s 159ms/step
Image: image13.jpg - Predicted Accuracy: 0.4576 - Severity: less severe
1/1 [==============================] - 0s 157ms/step
Image: image10.jpg - Predicted Accuracy: 0.2979 - Severity: less severe
1/1 [==============================] - 0s 159ms/step
Image: image7.jpg - Predicted Accur

In [12]:
#rcnn

from google.colab import drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import (
    Dense,
    GlobalAveragePooling2D,
    SimpleRNN,
    Dropout,
    Reshape,
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import os
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define main data directory
neuro_dir = '/content/drive/MyDrive/Neuro'

# Load pre-trained InceptionV3 model without top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Reshape for RCNN
rcnn_input = Reshape((1, 128))(x)  # Reshape to (batch_size, timesteps, features)

# Add RNN layer
rnn_layer = SimpleRNN(64)(rcnn_input)

predictions_layer = Dense(1, activation='sigmoid')(rnn_layer)
model = Model(inputs=base_model.input, outputs=predictions_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# List all main subfolders
main_subfolders = ['dhdleg', 'dhdhead', 'dhdgen', 'dhdpelvic','llg']

# Initialize counters for high severe and less severe images
high_severe_count = 0
less_severe_count = 0

# Lists to store true labels and predicted probabilities for evaluation
true_labels = []
predicted_probs = []

# Loop through main subfolders
for main_subfolder in main_subfolders:
    main_data_dir = os.path.join(neuro_dir, main_subfolder)

    subsubfolders = ['train', 'test']

    for subsubfolder in subsubfolders:
        data_dir = os.path.join(main_data_dir, subsubfolder)

        print(f"Processing subsubfolder: {main_subfolder}/{subsubfolder}")

        # List all image filenames in the subfolder
        image_filenames = [filename for filename in os.listdir(data_dir) if filename.lower().endswith(('.jpg', '.jpeg'))]

        if not image_filenames:
            print("No images found in the directory.")
            continue

        for image_filename in image_filenames:
            # Load the image, preprocess it, and make a prediction
            image_path = os.path.join(data_dir, image_filename)
            img = load_img(image_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values
            img_array = img_array.reshape((1,) + img_array.shape)

            true_label = 1  # Assuming all are high severe
            true_labels.append(true_label)

            prediction = model.predict(img_array)
            predicted_prob = prediction[0][0]
            predicted_probs.append(predicted_prob)

            if predicted_prob <= 0.40:
                less_severe_count += 1
            else:
                high_severe_count += 1

            severity = "less severe" if predicted_prob <= 0.80 else "high severe"

            print(f"Image: {image_filename} - Predicted Accuracy: {predicted_prob:.4f} - Severity: {severity}")

# Calculate overall accuracies
total_images = high_severe_count + less_severe_count
high_severe_accuracy = (high_severe_count / total_images) * 100 if total_images > 0 else 0
less_severe_accuracy = (less_severe_count / total_images) * 100 if total_images > 0 else 0

print(f"\nOverall High Severe Percentage: {high_severe_accuracy:.2f}%")
print(f"Overall Less Severe Percentage: {less_severe_accuracy:.2f}%")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing subsubfolder: dhdleg/train
1/1 [==============================] - 2s 2s/step
Image: image5.jpg - Predicted Accuracy: 0.5327 - Severity: less severe
1/1 [==============================] - 0s 150ms/step
Image: image21.jpg - Predicted Accuracy: 0.3669 - Severity: less severe
1/1 [==============================] - 0s 157ms/step
Image: image16.jpg - Predicted Accuracy: 0.3571 - Severity: less severe
1/1 [==============================] - 0s 140ms/step
Image: image8.jpg - Predicted Accuracy: 0.5250 - Severity: less severe
1/1 [==============================] - 0s 141ms/step
Image: image13.jpg - Predicted Accuracy: 0.6116 - Severity: less severe
1/1 [==============================] - 0s 138ms/step
Image: image10.jpg - Predicted Accuracy: 0.5567 - Severity: less severe
1/1 [==============================] - 0s 153ms/step
Image: image7.jpg - Predicted Accur